In [ ]:
import os
import requests  
import httpx  
import json  
from dotenv import load_dotenv
from IPython.display import display, Markdown, Image  
from openai import OpenAI  

from PIL import Image
import asyncio, pathlib
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")



In [ ]:
MCP_BASE = "http://localhost:7860/gradio_api/mcp/sse"

In [ ]:
from agents.mcp import MCPServerSse

In [ ]:
mcp_tool = MCPServerSse({
    "name": "AI Tutor",
    "url": MCP_BASE,
    "timeout": 30,
    "client_session_timeout_seconds":60
})

In [ ]:
client = httpx.Client() 
def fetch_schema(server_url):
    """Fetches and parses the MCP schema from a server."""
    
    schema_url = server_url.replace("/sse", "/schema")
    print(f"Fetching schema from: {schema_url}")
    
    response = client.get(schema_url, timeout = 10)  # Add a timeout
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    schema_data = response.json()
    print("Schema fetched successfully!")
    return schema_data

In [ ]:
print("--- Fetching AI Tutor Schema ---")
tutor_schema = fetch_schema(MCP_BASE)

if tutor_schema:
    print("\nAI Tutor Schema Contents:")
    # Pretty print the JSON manifest
    print(json.dumps(tutor_schema, indent=2))

print("\n" + "=" * 50 + "\n")

In [ ]:
from agents import Agent, Runner

agent = Agent(
    name = "Smart Assistant",
    instructions = """
    Context
    -------
    You are an AI assistant with access to an MCP server exposing **four streaming tools**:

    1. **explain_concept**  
    Arguments: { "question": <str>, "level": <int 1‑5> }  
    • Streams an explanation of any concept at the requested depth.

    2. **summarize_text**  
    Arguments: { "text": <str>, "compression_ratio": <float 0.1‑0.8> }  
    • Streams a concise summary ~compression_ratio × original length.

    3. **generate_flashcards**  
    Arguments: { "topic": <str>, "num_cards": <int 1‑20> }  
    • Streams JSON‑lines flashcards: one card per line `{ "q":…, "a":… }`.

    4. **quiz_me**  
    Arguments: { "topic": <str>, "level": <int 1‑5>, "num_questions": <int 1‑15> }  
    • Streams an MC‑question quiz, then an ANSWER KEY section.

    Objective
    ---------
    Help users learn by:
    • Explaining concepts at the depth they request.  
    • Summarising long passages.  
    • Generating flashcards for self‑study.  
    • Quizzing them interactively.

    How to respond
    --------------
    • For each user request, decide which tool (if any) fulfils it best.  
    • Call the tool via MCP by returning *only* the JSON with `"tool"` and `"arguments"` (no extra text).  
    • If a follow‑up conversation is needed (e.g., clarification), ask the user first.  
    • If no tool fits, answer directly in plain language.

    Examples
    --------
    User: “Explain quantum tunnelling like I’m 10.”  
    → Call `explain_concept` with { "question": "quantum tunnelling", "level": 2 }

    User: “Summarise this article to 20 %.” + <article text>  
    → Call `summarize_text` with { "text": "...", "compression_ratio": 0.2 }

    Chat capability
    ---------------
    After each tool call completes (streaming back to the user), remain in the chat loop ready for the next user turn.
    """,
    model = "gpt-4o-mini",
    mcp_servers = [mcp_tool],
)

In [ ]:
await mcp_tool.connect()  
result = None
while True:
    user_input = input("User: ")
    if user_input.lower() in {"exit", "quit"}:
        break
        
    if result is not None:
        new_input = result.to_input_list() + [{"role": "user", "content": user_input}]
    else:
        new_input = [{"role": "user", "content": user_input}]
    print("\nUser Input:")
    print_markdown(user_input)

    
   # This is the core AI agent execution step. It runs your agent with the new_input.

    result = await Runner.run(starting_agent = agent, input = new_input)
    print("\nAssistant:")
    print_markdown(result.final_output)

In [ ]:
for i in result.to_input_list():
    for key in i.keys():
        if key == 'arguments':
            print("Tool: ", i['name'])
            print("Arguments: ", i['arguments'])